Depending on how you set up your env you might need to run the following pip

In [ ]:
!pip install dash-bootstrap-components

In [ ]:
!pip install jupyter_dash

In [134]:
import plotly.express as px
import pandas as pd
import numpy as np
import json
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from jupyter_dash import JupyterDash
import dash_html_components as html
import dash_core_components as dcc
import dash_bootstrap_components as dbc
from dash.dependencies import Output, Input
from dash.exceptions import PreventUpdate

In [147]:
canadaData = pd.read_csv("13100096.csv")
provinceCapital = pd.read_csv("provinceCapital.csv")
notes = pd.read_csv('notes.csv')
notesIndicator = pd.read_csv('notesIndicators.csv')
with open('canada_provinces.geojson') as f:
    provinces = json.load(f)
provincialData = canadaData[ canadaData['GEO'] != 'Canada (excluding territories)' ]
merged_df = pd.merge(provincialData,provinceCapital,left_on="GEO",right_on="Province")
merged_df["Hover_Label"] = merged_df["Capital"] + ", " + merged_df["Province"]

In [136]:
ageDict = { idxAge : canadaData['Age group'].unique()[idxAge]  for idxAge in range( canadaData['Age group'].nunique() ) }
sexDict = {idxSex:   canadaData['Sex'].unique()[idxSex]  for idxSex in range( canadaData['Sex'].nunique() ) }
unitsDict = {idxUnit:  canadaData['UOM'].unique()[idxUnit]  for idxUnit in range( canadaData['UOM'].nunique() ) }
yearDict = {0: 2015, 1: 2016, 2: 2017, 3: 2018, 4: 2019, 5: 2020}
variable_clustering = ['None', 'GDP 2020', 'Population', 'GPD per capita']

In [138]:
def getNotes(indicator):
    references = notesIndicator[notesIndicator['Indicator']==indicator]['Reference'].values[0]
    refAtoms = references.split(';')
    noteList = []
    for note in refAtoms:
        noteList.append( notes[ notes['Note ID']== int(note) ]['Note'].values[0] )
    return noteList

In [159]:
app = JupyterDash(__name__, 
                external_stylesheets=[dbc.themes.BOOTSTRAP]) 

app.layout = html.Div([ 
    html.H1('Canada Health dataset', style={'textAlign': 'center'}),
    html.Br(),
    dbc.Row([
        dbc.Col([dbc.Label("Indicator:", style={'textAlign': 'left', 'fontSize': '25px', 'fontWeight': 'bold'}),
                 dcc.Dropdown(id='id_indicator', value="Perceived mental health, fair or poor",
                              options=[{'label':ind, 'value':ind } for ind in canadaData["Indicators"].unique()])], md=12, lg=12),
            ]),
    
    html.Br(),
    
    dbc.Row([
        dbc.Col([dbc.Label("Age group:"),
                        dcc.Slider(id = 'id_age',
                        min = 0,
                        max = 5,
                        step=None,
                        dots=True,
                        value = 0,
                        marks= ageDict,
                        included = False)   
                        ], md=5, lg=5),
        
        dbc.Col([dbc.Label("Sex group:"),
                        dcc.Slider(id='id_sex',
                        min=0,
                        max=2,
                        step=None,
                        dots=True,
                        value = 0,
                        marks = sexDict,
                            included=False)
                        ], md = 3, lg = 3),
        
        dbc.Col([dbc.Label("Units:"),
                        dcc.Slider(id = "id_units",
                        min=0,
                        max=1,
                        step=None,
                        dots=True,
                        value = 0,
                        marks = unitsDict,
                            included=False)
                        ], md = 2, lg = 2)   
                ]),
    
    dbc.Row([
        dbc.Col([dcc.Graph("plot_one_choropleth")], md = 6, lg = 6),
        dbc.Col([dcc.Graph("plot_two_scatterplot")], md = 6, lg = 6)
            ]),
    
    dbc.Row([dcc.Markdown(id='indicator_map_details_md')]),
    
    dbc.Row([dbc.Col([dbc.Label("Select the year:"),
                      dcc.Slider(id = 'id_year',
                                 min = 0,
                                 max = 5,
                                 step=None,
                                 dots=True,
                                 value = 0,
                                 marks= {0: {'label':'2015'},
                                         1: {'label':'2016'},
                                         2: {'label':'2017'},
                                         3: {'label':'2018'},
                                         4: {'label':'2019'},
                                         5: {'label':'2020'}},
                                 included = False)], md=4, lg=4),
            
            dbc.Col([dbc.Label("Select the number of clusters:"),
                     dcc.Slider(id='id_clusters',
                                min=2,
                                max=5,
                                step=None,
                                dots=True,
                                value = 2,
                                marks= { int(bins) : { 'label': str( int(bins) ) } for bins in np.arange(2,6)}, 
                                included=False)], md = 4, lg = 4),
            
            dbc.Col([dbc.Label("Select an additional variable for clustering:"),
                     dcc.Dropdown(id = "id_variable",
                                  value = "None",
                                  options= [ {'label':variable, 'value':variable } for variable in variable_clustering]),], md = 4, lg = 4)                  
            ]),
    
    dbc.Row([dcc.Graph("plot_three_clusterplot")]),
                        
                        ])

# CHROPLETH ONE ###########################################################################
@app.callback(Output(component_id="plot_one_choropleth", component_property="figure"),
              Input(component_id="id_indicator", component_property="value"),
              Input(component_id="id_age", component_property="value"),
              Input(component_id="id_sex", component_property="value"),
              Input(component_id="id_units", component_property="value"))

def plot_choropleth(indicator, age, sex, units):
        
        ageDict1 = ageDict[age]
        sexDict1 = sexDict[sex]
        
        if units==0:
                charValue = 'Number of persons'
        else:
                charValue = 'Percent'

        
        df = (provincialData[(provincialData['Age group'] ==  ageDict1) &
                             (provincialData['Sex'] ==  sexDict1) & 
                             (provincialData['Characteristics'] == charValue) & 
                             (provincialData['Indicators'] == indicator)] ) 
        
        fig = px.choropleth(df,
                          geojson=provinces,
                          featureidkey='properties.name',
                          locations='GEO',
                          color='VALUE',
                          color_continuous_scale='Inferno',
                          title=indicator,
                          height=600,
                          scope='north america',
                          animation_frame='REF_DATE')
        
        fig.layout.geo.showframe = False
        fig.update_layout(coloraxis_colorbar_title_text = 'VALUE')
        fig.layout.geo.landcolor = 'white'
        fig.update_geos(fitbounds="locations")
        
        return fig

# SCATTERPLOT & MARKDOWN ######################################################################  
@app.callback(Output(component_id="plot_two_scatterplot", component_property="figure"),
              Output(component_id='indicator_map_details_md', component_property="children"),  
              Input(component_id="id_indicator", component_property="value"),
              Input(component_id="id_age", component_property="value"),
              Input(component_id="id_sex", component_property="value"),
              Input(component_id="id_units", component_property="value"))

def plot_scatterplot(indicator, age, sex, units):
        
        ageDict1 = ageDict[age]
        sexDict1 = sexDict[sex]
        
        if units==0:
                charValue = 'Number of persons'
        else:
                charValue = 'Percent'

        df = (merged_df[(merged_df['Age group'] ==  ageDict1) & 
                        (merged_df['Sex'] ==  sexDict1) &
                        (merged_df['Characteristics'] == charValue) &
                        (merged_df['Indicators'] == indicator)] )

        fig2 = px.scatter_mapbox(df,
                                 lon = 'Longitude',
                                 lat = 'Latitude',
                                 zoom = 2,
                                 size = 'VALUE', 
                                 size_max = 75,
                                 color='Region',
                                 animation_frame='REF_DATE',
                                 opacity=0.65,
                                 height=600,
                                 hover_data=['Region'],
                                 color_discrete_sequence=px.colors.qualitative.G10,
                                 category_orders=dict(Region=["Atlantic", "Prairies", "West", "Central"]),
                                 mapbox_style='stamen-toner',
                                 hover_name= df['Hover_Label'],
                                 title="")
        
        #MARKDOWN
        notes = getNotes(indicator)
        markdown = "## "+indicator+":\n"
    
        for note in notes:
                markdown+="* "+note+"\n"

        return fig2, markdown

    
# KNN CLUSTER PLOT ###################################################################### 
@app.callback(Output(component_id="plot_three_clusterplot", component_property="figure"),
              Input(component_id="id_indicator", component_property="value"),
              Input(component_id="id_age", component_property="value"),
              Input(component_id="id_sex", component_property="value"),
              Input(component_id="id_units", component_property="value"),
              Input(component_id="id_year", component_property="value"),
              Input(component_id="id_clusters", component_property="value"),
              Input(component_id="id_variable", component_property="value"))

def plot_map_cluster(indicator, age, sex, units, year, clusters, cvar):
        
        variable = ["VALUE"]
        if not cvar == "None":
                variable.append(cvar)

        ageDict1 = ageDict[age]
        sexDict1 = sexDict[sex]
        yearDict1 = yearDict[year]
        
        if units==0:
                charValue = 'Number of persons'
        else:
                charValue = 'Percent'

        df = provincialData.merge(provinceCapital, left_on="GEO", right_on="Province", how="left")
        
        df1 =   df[ (df['Age group'] ==  ageDict1 ) 
                     & (df['Sex'] ==  sexDict1 ) 
                     & (df['Characteristics'] == charValue) 
                     & (df['Indicators'] == indicator)  
                     & (df["REF_DATE"] == yearDict1 )]

        arrayData = df1[variable]
        if arrayData.isna().all().any():
                return px.choropleth(title='No available data for the selected combination of year/indicators.')
        
        imputeData = SimpleImputer(missing_values=np.nan, strategy='mean').fit_transform( arrayData ) # imputing
        scaledData = StandardScaler().fit_transform( imputeData ) #scaling
        kmeans = KMeans(n_clusters=clusters)
        kmeans.fit(scaledData)

        fig3 = px.choropleth(df1,
                             geojson=provinces,
                             featureidkey='properties.name',
                             locations = "GEO",
                             scope="north america",
                             color=[str(x) for x in kmeans.labels_],
                             hover_data=variable,
                             height=650,
                             title=f'{yearDict1}. Number of clusters: {clusters}<br>Inertia: {kmeans.inertia_:,.2f}',
                             color_discrete_sequence=px.colors.qualitative.T10 )
        
        fig3.layout.geo.showframe = False
        fig3.update_layout(coloraxis_colorbar_title_text = 'VALUE')
        fig3.layout.geo.landcolor = 'white'
        fig3.update_geos(fitbounds="locations")

        return fig3           
        

app.run_server(mode='external', height=600, width='85%', debug=False)

Dash app running on http://127.0.0.1:8050/
